In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [ ]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd

# Get the current datetime64[us, Etc/UTC]
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features_and_target
features, _ = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

2025-02-22 17:08:29,796 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:08:29,796 INFO: Initializing external client
2025-02-22 17:08:29,796 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:08:30,609 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Fetching data from 2025-01-24 22:08:29.796787+00:00 to 2025-02-22 21:08:29.796787+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.86s) 


In [9]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-22 17:08:44,214 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:08:44,225 INFO: Initializing external client
2025-02-22 17:08:44,225 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:08:45,117 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [10]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [11]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,67,0.0,2025-02-22 23:00:00+00:00
1,13,15.0,2025-02-22 23:00:00+00:00
2,65,1.0,2025-02-22 23:00:00+00:00
3,69,0.0,2025-02-22 23:00:00+00:00
4,155,0.0,2025-02-22 23:00:00+00:00
...,...,...,...
252,98,0.0,2025-02-22 23:00:00+00:00
253,79,68.0,2025-02-22 23:00:00+00:00
254,25,3.0,2025-02-22 23:00:00+00:00
255,242,1.0,2025-02-22 23:00:00+00:00


In [13]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-02-22 17:11:51,395 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:11:51,404 INFO: Initializing external client
2025-02-22 17:11:51,405 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:11:52,530 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [14]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1212635/fs/1200268/fg/1402204


Uploading Dataframe: 100.00% |██████████| Rows 257/257 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1212635/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)